In [2]:
import logging
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import linregress
import math
import io
import gzip
import statsmodels.api as sm
from sklearn.decomposition import PCA
import scvi
import scipy as sp
import os
import anndata as ad
import matplotlib.gridspec as gridspec
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, r2_score, f1_score
import statsmodels.formula.api as smf
from sklearn.linear_model import Ridge, LinearRegression
from matplotlib.colors import ListedColormap
import pickle


import sys
import os
from contextlib import contextmanager

@contextmanager
def suppress_stdout_stderr():
    with open(os.devnull, 'w') as devnull:
        old_stdout = sys.stdout
        old_stderr = sys.stderr
        try:
            sys.stdout = devnull
            sys.stderr = devnull
            yield
        finally:
            sys.stdout = old_stdout
            sys.stderr = old_stderr


In [20]:
#Load the Siletti et al Human Brain Atlas neuron data
siletti_humanbrain = sc.read("/home/t-ajaynadig/perturbation_project/ldvae_hematopoietic/training_data_raw/SilettiBrainAtlas/Neurons.h5ad",backed = "r+")

In [21]:
#subset to donor_id == "H18.30.002", and simulataneously sample 10k cells, only indexing the backed object once, i.e. by creating one filter. Do not use copy() as it will load the entire dataset into memory
#get the indices for the correct donor
donor_indices = siletti_humanbrain.obs["donor_id"] == "H18.30.002"
#sample 10k of these indices
sampled_indices = np.random.choice(np.where(donor_indices)[0],10000,replace = False)
#subset the data
siletti_humanbrain = siletti_humanbrain[sampled_indices,:]

In [22]:
siletti_humanbrain.obs['supercluster_term'].value_counts()

supercluster_term
Upper-layer intratelencephalic       2111
Splatter                             1103
Deep-layer intratelencephalic         962
MGE interneuron                       857
CGE interneuron                       818
Medium spiny neuron                   674
Amygdala excitatory                   542
Upper rhombic lip                     408
Midbrain-derived inhibitory           403
Thalamic excitatory                   350
Hippocampal CA1-3                     322
Deep-layer corticothalamic and 6b     279
Lower rhombic lip                     266
Hippocampal dentate gyrus             235
Eccentric medium spiny neuron         175
LAMP5-LHX6 and Chandelier             151
Miscellaneous                         125
Deep-layer near-projecting             79
Mammillary body                        64
Hippocampal CA4                        38
Cerebellar inhibitory                  38
Name: count, dtype: int64

In [23]:
#write out subset to /home/t-ajaynadig/perturbation_project/ldvae_hematopoietic/eval_data/SilettiNeuron_10Ksubset/
siletti_humanbrain.write("/home/t-ajaynadig/perturbation_project/ldvae_hematopoietic/eval_data/SilettiNeuron_10Ksubset/Neurons_H18.30.002_10Ksubset.h5ad")